<a href="https://colab.research.google.com/github/Angelvj/Alzheimer-disease-classification/blob/main/code/phase_1_deep_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
kaggle = False

# Imports

In [ ]:
import shutil
import sys
import tensorflow as tf

if tf.io.gfile.exists('Alzheimer-disease-classification'):
    shutil.rmtree('Alzheimer-disease-classification')
! git clone https://github.com/Angelvj/Alzheimer-disease-classification.git

sys.path.insert(0,'/content/Alzheimer-disease-classification/code')

In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold
from google.colab import drive

import functions.lr_schedules as lr_schedules
import functions.io_utils as io
from functions.tfrec_loading import get_dataset
from functions.model_evaluation import repeated_kfold, plot_epochs_history, get_rkf_history, get_predictions, calculate_results, train_model, present_results
from models import feedforward_models_pet, feedforward_models_mri

# Hardware config.

In [ ]:
DEVICE = 'TPU' # or GPU
tpu = None

if DEVICE == 'TPU':
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        STRATEGY = tf.distribute.experimental.TPUStrategy(tpu)
    except ValueError:
        print('Could not connect to TPU, setting default strategy')
        tpu = None
        STRATEGY = tf.distribute.get_strategy()
elif DEVICE == 'GPU':
    STRATEGY = tf.distribute.MirroredStrategy()
    
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = STRATEGY.num_replicas_in_sync

print(f'Number of accelerators: {REPLICAS}')

# Constants

In [ ]:
SEED = 268 # Arbitrary seed

# Name of different datasets used
TFREC_DATASETS = ['tfrec-pet-spatialnorm-elastic-maxintensitynorm',
                  'tfrec-mri_grey-standarized-downscale',
                  'tfrec-mri_grey-standarized'
                 ]

# Shape of images on each dataset
SHAPES = [(79, 95, 68, 1), (75, 90, 75, 1), (121, 145, 121, 1)]

# Number of repetitions and folds for repeated k-fold
REPS = 1
FOLDS = 2

# Different classes on the dataset
CLASSES = ['NOR', 'AD', 'MCI']

# Path to data
if kaggle:
    INPUT_DATAPATH = '/kaggle/input/' if tpu is None else None
    METADATA_PATH = '/kaggle/input/'
else:
    drive.mount('/content/drive') 
    INPUT_DATAPATH = '/content/drive/MyDrive/data/'
    METADATA_PATH = '/content/drive/MyDrive/data/'


# Function for setting up constant for a given dataset
def select_dataset(ds_id):
    global DS, IMG_SHAPE, DS_PATH, INPUT_DATAPATH, X_train, y_train, X_test, y_test
    DS = TFREC_DATASETS[ds_id]
    IMG_SHAPE = SHAPES[ds_id]
    if INPUT_DATAPATH == None:
        user_secrets = UserSecretsClient()
        user_credential = user_secrets.get_gcloud_credential()
        user_secrets.set_tensorflow_credential(user_credential)
        DS_PATH = KaggleDatasets().get_gcs_path(DS)
    else:
        DS_PATH = INPUT_DATAPATH + DS

    # Load filenames and labels
    metadata_train = pd.read_csv(METADATA_PATH + DS + '/train/train_summary.csv', encoding='utf-8')
    metadata_test = pd.read_csv(METADATA_PATH + DS + '/test/test_summary.csv', encoding='utf-8')
    X_train = DS_PATH + '/train/' + metadata_train.iloc[:, 0].to_numpy()
    y_train = np.argmax(metadata_train.iloc[:,-len(CLASSES):].to_numpy(), axis=1)
    X_test = DS_PATH + '/test/' + metadata_test.iloc[:, 0].to_numpy()
    y_test = np.argmax(metadata_test.iloc[:,-len(CLASSES):].to_numpy(), axis=1)

#  Experiments execution (only final version)

## PET

In [6]:
select_dataset(0) # Preprocessed PET dataset

### Exp.1 --> 1 conv. layer

In [ ]:
BATCH_SIZE = 8
EPOCHS = 50
LR = 0.00001
DECAY_RATE = 0.1

detailed_history = repeated_kfold(feedforward_models_pet.model_0, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY_RATE, EPOCHS)])
# save_dict(detailed_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-0_results.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-0.txt')

### Exp.2 --> 2 conv. layer

In [ ]:
BATCH_SIZE = 4
EPOCHS = 50
LR = 0.00001
DECAY_RATE = 0.1

detailed_history = repeated_kfold(feedforward_models_pet.model_1, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY_RATE, EPOCHS)])
# save_dict(detailed_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-1_results.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-1.txt')

### Exp.3 --> 3 conv. layers

In [ ]:
BATCH_SIZE = 8
EPOCHS = 50
LR = 0.0001
DECAY_RATE = 0.01

detailed_history = repeated_kfold(feedforward_models_pet.model_2, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY_RATE, EPOCHS)])
# save_dict(detailed_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-2_results.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-2.txt')

### Exp.4 --> 6 conv. layers

In [ ]:
BATCH_SIZE = 4
EPOCHS = 40
LR = 0.000001
DECAY_RATE = 0.1

detailed_history = repeated_kfold(feedforward_models_pet.model_3, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY_RATE, EPOCHS)])
# save_dict(detailed_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-3_results.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-3.txt')

### Exp.5 --> 8 conv. layers

In [ ]:
BATCH_SIZE = 4
EPOCHS = 50
LR = 0.00001
DECAY_RATE = 0.1

detailed_history = repeated_kfold(feedforward_models_pet.model_4, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY_RATE, EPOCHS)])
# save_dict(detailed_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-4_results.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-4.txt')

# Train and get results on test data
model_4 = train_model(feedforward_models_pet.model_4, X_train, IMG_SHAPE, STRATEGY,
                      AUTO, BATCH_SIZE, EPOCHS, cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY_RATE, EPOCHS)])
y_pred = get_predictions(model_4, X_test, IMG_SHAPE, 3, AUTO, BATCH_SIZE)
results = calculate_results(y_test, y_pred)
present_results(results, {0:'NOR', 1:'AD', 2:'MCI'})
io.save_dict(results, 'model_4-pet-results.txt')

## MRI

In [10]:
select_dataset(1) # Preprocessed MRI dataset, downscaled

### Exp.1 --> 1 conv. layers

In [ ]:
BATCH_SIZE = 4
EPOCHS = 50
LR = 0.000001
DECAY_RATE = 0.1

detailed_history = repeated_kfold(feedforward_models_mri.model_0, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY_RATE, EPOCHS)])
# save_dict(detailed_history, 'mri-grey-standarized-downscale_model-0.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'mri-grey-standarized-downscale_model-0.txt')

### Exp.2 --> 3 conv. layers

In [ ]:
BATCH_SIZE = 4
EPOCHS = 70
LR = 0.00001
DECAY_RATE = 0.1

detailed_history = repeated_kfold(feedforward_models_mri.model_1, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY_RATE, EPOCHS)])
# save_dict(detailed_history, 'mri-grey-standarized-downscale_model-1.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'mri-grey-standarized-downscale_model-1.txt')

### Exp.3 --> 4 conv. layers

In [ ]:
BATCH_SIZE = 4
EPOCHS = 50
LR = 0.000001
DECAY_RATE = 0.1

detailed_history = repeated_kfold(feedforward_models_mri.model_2, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY_RATE, EPOCHS)])
# save_dict(detailed_history, 'mri-grey-standarized-downscale_model-2.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'mri-grey-standarized-downscale_model-2.txt')

### Exp.4 --> 6 conv. layers

In [ ]:
BATCH_SIZE = 4
EPOCHS = 50
LR = 0.000001
DECAY_RATE = 0.1

detailed_history = repeated_kfold(feedforward_models_mri.model_3, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY_RATE, EPOCHS)])
# save_dict(detailed_history, 'mri-grey-standarized-downscale_model-3.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'mri-grey-standarized-downscale_model-3.txt')

### Exp.5 --> 9 conv. layers

In [ ]:
select_dataset(2) # Preprocessed MRI dataset

BATCH_SIZE = 4
EPOCHS = 1
LR = 0.000001
DECAY_RATE = 0.1

detailed_history = repeated_kfold(feedforward_models_mri.model_4, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY_RATE, EPOCHS)])
# save_dict(detailed_history, 'mri-grey-standarized-downscale_model-4.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'mri-grey-standarized-downscale_model-4.txt')

# Train and get results on test data
model_4 = train_model(feedforward_models_mri.model_4, X_train, IMG_SHAPE, STRATEGY,
                      AUTO, BATCH_SIZE, EPOCHS, cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY_RATE, EPOCHS)])
y_pred = get_predictions(model_4, X_test, IMG_SHAPE, 3, AUTO, BATCH_SIZE)
results = calculate_results(y_test, y_pred)
present_results(results, {0:'NOR', 1:'AD', 2:'MCI'})
io.save_dict(results, 'model_4-mri-results.txt')